# 인스타 데이터 수집_v1
#### 가져와야 할 외부지표:  
ID, 게시물 수, 팔로워 수, 팔로우 수 (프로필 정보 관련)  
좋아요 수, 댓글 수, 게시글 내용, 댓글 내용 (각 게시물 정보 관련)

### 코드 실행 시 주의!!
- 크롬 드라이버를 다운받고 이 코드 파일와 동일한 경로에 넣어 주세요.  
- 이 코드 창의 브라우저 크기를 줄이고 코드 실행해야 잘 작동됩니다.  

## 1. 세팅

In [ ]:
# !pip install selenium
# !pip install beautifulsoup4
# !pip install webdriver_manager

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # 크롬 드라이버 자동 업데이트을 위한 모듈

import urllib.request # 이미지 저장 라이브러리

In [ ]:
# 크롬 드라이버 옵션 관련 코드

# 브라우저 꺼짐 방지 옵션
options = Options()
options.add_experimental_option("detach", True)
options.add_argument('window-size=1920x1080') # pc용 사이즈
options.add_argument('--start-maximized')     # 브라우저가 최대화된 상태로 실행

# 불필요한 에러 메시지 삭제
options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 크롬 드라이버 최신 버전 설정
service = Service(executable_path=ChromeDriverManager().install())

driver = webdriver.Chrome(service=service, options=options)

driver 객체의 page_source 인스턴스 변수:  
웹 서버로부터 전달받은 HTML 코드가 저장되어 있음.  

1. selenium에서 page_source를 사용하여 해당 페이지의 소스, html을 가져오고 
2. 가져온 html을 BeautifulSoup으로 넘겨줌.
   
이를 통해 페이지 이동 및 동작은 selenium으로 크롬 드라이버를 조작하여 할 수 있고,  
원하는 페이지로 이동후 html을 BeatifulSoup으로 전달하여 원하는 웹 데이터를 가져와 사용할 수 있다.

In [ ]:
# 인스타그램 접속
driver.get('https://instagram.com')

# 인스타그램 자동 로그인
driver.implicitly_wait(10) # 나타날 때까지 최대 10초동안 기다리기
login_id = driver.find_element(By.CSS_SELECTOR, 'input[name="username"]')
login_id.send_keys('본인의 인스타 아이디 입력')
driver.implicitly_wait(10)
login_pwd = driver.find_element(By.CSS_SELECTOR, 'input[name="password"]')
driver.implicitly_wait(10)
login_pwd.send_keys('본인의 인스타 비밀번호 입력')
driver.implicitly_wait(10)
login_id.send_keys(Keys.ENTER)
time.sleep(3)

In [ ]:
# 샘플 url
url = "원하는 인플루언서의 주소 입력"

## 2. 프로필 관련 정보: 팔로워, 팔로잉, 게시물 수, 닉네임, 자기소개
- bs4로 html문서를 읽어 수집

In [ ]:
# 프로필 관련 정보 가져오기
driver.get(url)           # 페이지 이동
time.sleep(3)

html = driver.page_source # html(페이지 소스) 가져오기
soup = BeautifulSoup(html)

In [ ]:
profile = soup.select("._ac2a > span")
profile
# 각각 게시물 수, 팔로워 수, 팔로우 수 

In [ ]:
media_count = profile[0].text     # 게시물 수
followers_count = profile[1].text # 팔로워 수
follows_count = profile[2].text   # 팔로우 수 

In [ ]:
print(media_count)
print(followers_count)
print(follows_count)

In [ ]:
try: 
    nickname = soup.select_one("._aacy").text     # 닉네임
    print(nickname)
except:
    print("닉네임 없음")

In [ ]:
try: 
    introduction = soup.select_one("._aacx").text # 자기소개
    print(introduction)
except:
    print("자기소개 없음")

## 3. 인스타그램 계정의 각 게시물의 대표 이미지와 좋아요 수
-  셀레니움으로 게시물을 클릭해 이동하며 수집

#### 좋아요 개수 추출에서 생기는 문제:
인플루언서가 좋아요 수를 공개하지 않으면 선택자 명과 태그 구조가 달라집니다.  
최대한 타협안으로 좋아요 개수 전체 태그의 클래스 명을 사용했습니다.  
그래서 좋아요 개수뿐만 아니라 글자도 같이 포함되는데 이는 파이썬 전처리로 해결해야 될 것 같습니다.  

In [ ]:
# 인스타그램 계정의 각 게시물의 대표 이미지와 좋아요 수 가져오기

# 첫번째 게시물 클릭
driver.find_element(By.CSS_SELECTOR, '._aagu').click()
driver.implicitly_wait(10)


# 첫번째 게시물부터 데이터를 추출하고 다음 게시물을 클릭
for i in range(5): # 이 숫자를 조절하면 원하는 게시물 수만큼 가져올 수 있음.
    
    # 예외처리로 일단 사진만 가져오게 하고 영상은 지나치도록 함.
    try:
        # 대표 이미지 가져오기
        img_element = driver.find_element(By.CSS_SELECTOR, "._aagu._aato img").get_attribute("src")
        print(f'{i}번째 게시물 이미지: ' + img_element)

        # 좋아요 수
        like_element = driver.find_element(By.CSS_SELECTOR, "._ae5m .x193iq5w.xeuugli").text # 좋아요 수 영역
        print(f'{i}번째 게시물 좋아요 수: ' + like_element)

        driver.find_element(By.CSS_SELECTOR, '._aaqg._aaqh').click() # 다음 버튼 클릭
        driver.implicitly_wait(2)

    except:
        # 영상은 지나치기 
        print(f'{i}번째 게시물' + "은 video입니다.")
        
        # 좋아요 수
        like_element = driver.find_element(By.CSS_SELECTOR, "._ae5m .x193iq5w.xeuugli").text
        print(f'{i}번째 게시물 좋아요 수: ' + like_element)

        driver.find_element(By.CSS_SELECTOR, '._aaqg._aaqh').click() # 다음 버튼 클릭
        driver.implicitly_wait(2)

    print()

In [ ]:
# 참고
# 좋아요 개수 공개했을 때 사용할 선택자 명: "._ae5m .html-span"
# 좋아요 개수 공개하지 않을 때 사용할 선택자 명: "._ae5m a:nth-child(2)"
# 이 코드에서는 "._ae5m .x193iq5w.xeuugli"를 사용했습니다.